<a href="https://colab.research.google.com/github/flatneuron/football-player-reidentification/blob/main/reid_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torchreid

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.7/92.7 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torchreid: filename=torchreid-0.2.5-py3-none-any.whl size=144324 sha256=a460651519913b2970e4700b28c61a538fad48ec9c00f7fe5e1c3fcf68d36289
  Stored in directory: /root/.cache/pip/wheels/61/dc/08/b478469bab07b5ede9e962968ebe3c8961c10c5fc106a6c697
Successfully built torchreid


In [ ]:
import os
import os.path as osp
from torchreid.reid.data.datasets.dataset import ImageDataset

class MyReidDataset(ImageDataset):
    dataset_dir = 'drive/MyDrive/stealth mode/reid-data_mine'

    def __init__(self, root='', **kwargs):
        # Resolve absolute paths
        self.root = osp.abspath(osp.expanduser(root))
        self.dataset_dir = osp.join(self.root, self.dataset_dir)

        # 1) collect all camera-folder names and person-folder names
        splits = ["train", "query", "gallery"]
        cam_names = set()
        pid_names = set()

        for split in splits:
            split_dir = osp.join(self.dataset_dir, split)
            if not osp.isdir(split_dir):
                raise ValueError(f"Expected '{split_dir}' to exist.")
            for cam in os.listdir(split_dir):
                cam_dir = osp.join(split_dir, cam)
                if not osp.isdir(cam_dir):
                    continue
                cam_names.add(cam)
                for pid in os.listdir(cam_dir):
                    pid_dir = osp.join(cam_dir, pid)
                    if osp.isdir(pid_dir):
                        pid_names.add(pid)

        # 2) build zero-based mappings
        cam_list = sorted(cam_names)
        pid_list = sorted(pid_names)
        cam2label = {cam: idx for idx, cam in enumerate(cam_list)}
        pid2label = {pid: idx for idx, pid in enumerate(pid_list)}

        # 3) helper to parse each split into (img_path, pid, camid)
        def parse_split(split):
            data = []
            split_dir = osp.join(self.dataset_dir, split)
            for cam in os.listdir(split_dir):
                cam_dir = osp.join(split_dir, cam)
                if not osp.isdir(cam_dir):
                    continue
                camid = cam2label[cam]
                for pid in os.listdir(cam_dir):
                    pid_dir = osp.join(cam_dir, pid)
                    if not osp.isdir(pid_dir):
                        continue
                    pidid = pid2label[pid]
                    # collect all image files under this pid folder
                    for fname in os.listdir(pid_dir):
                        if not fname.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp')):
                            continue
                        img_path = osp.join(pid_dir, fname)
                        data.append((img_path, pidid, camid))
            return data

        # 4) generate the three splits
        train = parse_split("train")
        query = parse_split("query")
        gallery = parse_split("gallery")

        def check_pid_consistency(query, gallery):
          query_pids = set([pid for _, pid, _ in query])
          gallery_pids = set([pid for _, pid, _ in gallery])

          missing = query_pids - gallery_pids
          if missing:
              print("🚫 Missing PIDs in gallery:", missing)
          else:
              print("✅ All query PIDs are present in gallery!")

        # Call it like this
        check_pid_consistency(query, gallery)

        # 5) pass to super
        super(MyReidDataset, self).__init__(train, query, gallery, **kwargs)


/usr/local/lib/python3.11/dist-packages/torchreid/reid/metrics/rank.py:11: UserWarning: Cython evaluation (very fast so highly recommended) is unavailable, now use python evaluation.
  warnings.warn(


In [ ]:
import torchreid
torchreid.reid.data.datasets.register_image_dataset('MY-reid-dataset1', MyReidDataset)
ROOT_DIR = ''
datamanager = torchreid.data.ImageDataManager(
    root=ROOT_DIR,
    sources='MY-reid-dataset1',
    transforms=None,
    workers=2,
    height=256,
    width=256
)


######################################
# You will get the following details as output
# Building train transforms ...
# + resize to 256x256
# + to torch tensor of range [0, 1]
# + normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# Building test transforms ...
# + resize to 256x256
# + to torch tensor of range [0, 1]
# + normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# => Loading train (source) dataset
# => Loaded Pigs
#   -------------------------------------------
#   subset   | # ids | # tracklets | # cameras
#   -------------------------------------------
#   train    |     3 |        1227 |         4
#   query    |     3 |          48 |         4
#   gallery  |     3 |        1255 |         4
#   -------------------------------------------


#   **************** Summary ****************
#   source             : ['pigs']
#   # source datasets  : 1
#   # source ids       : 3
#   # source tracklets : 1227
#   # source cameras   : 4
#   target             : ['pigs']
#   *****************************************

Building train transforms ...
+ resize to 256x256
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x256
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
✅ All query PIDs are present in gallery!
=> Loaded MyReidDataset
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |    19 |     1325 |         1
  query    |    19 |      276 |         1
  gallery  |    19 |      305 |         1
  ----------------------------------------
=> Loading test (target) dataset
✅ All query PIDs are present in gallery!
=> Loaded MyReidDataset
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |    19 |     1325 |         1
  query    |    19 |     

In [ ]:
from torchreid.reid.models import build_model
import torch
model = build_model(
    name='osnet_x1_0',                # any key from __model_factory
    num_classes=datamanager.num_train_pids,  # number of training IDs (ReID classes)
    loss='softmax',                   # or 'triplet', or both as 'softmax+triplet'
    pretrained=True,                  # use ImageNet pretrained weights
    use_gpu=torch.cuda.is_available()  # move model to GPU if available
)


Downloading...
From: https://drive.google.com/uc?id=1LaG1EJpHrxdAxKnSCJ_i0u-nbxSAeiFY
To: /root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth
100%|██████████| 10.9M/10.9M [00:00<00:00, 112MB/s]

Successfully loaded imagenet pretrained weights from "/root/.cache/torch/checkpoints/osnet_x1_0_imagenet.pth"
** The following layers are discarded due to unmatched keys or layer size: ['classifier.weight', 'classifier.bias']


In [ ]:
optimizer = torchreid.optim.build_optimizer(
    model,
    optim='adam',
    lr=0.0003
)

In [ ]:
scheduler = torchreid.optim.build_lr_scheduler(
    optimizer,
    lr_scheduler='single_step',
    stepsize=20
)


In [ ]:
import torchreid
ROOT_DIR = ''
datamanager = torchreid.data.ImageDataManager(
    root=ROOT_DIR,
    sources='MY-reid-dataset1',
    transforms=None,
    workers=2,
    height=256,
    width=256
)


######################################
# You will get the following details as output
# Building train transforms ...
# + resize to 256x256
# + to torch tensor of range [0, 1]
# + normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# Building test transforms ...
# + resize to 256x256
# + to torch tensor of range [0, 1]
# + normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# => Loading train (source) dataset
# => Loaded Pigs
#   -------------------------------------------
#   subset   | # ids | # tracklets | # cameras
#   -------------------------------------------
#   train    |     3 |        1227 |         4
#   query    |     3 |          48 |         4
#   gallery  |     3 |        1255 |         4
#   -------------------------------------------


#   **************** Summary ****************
#   source             : ['pigs']
#   # source datasets  : 1
#   # source ids       : 3
#   # source tracklets : 1227
#   # source cameras   : 4
#   target             : ['pigs']
#   *****************************************

Building train transforms ...
+ resize to 256x256
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
Building test transforms ...
+ resize to 256x256
+ to torch tensor of range [0, 1]
+ normalization (mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
=> Loading train (source) dataset
✅ All query PIDs are present in gallery!
=> Loaded MyReidDataset
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |    19 |     1325 |         1
  query    |    19 |      276 |         1
  gallery  |    19 |      305 |         1
  ----------------------------------------
=> Loading test (target) dataset
✅ All query PIDs are present in gallery!
=> Loaded MyReidDataset
  ----------------------------------------
  subset   | # ids | # images | # cameras
  ----------------------------------------
  train    |    19 |     1325 |         1
  query    |    19 |     

In [ ]:
import torchreid
engine = torchreid.reid.engine.image.ImageSoftmaxEngine(
    datamanager,
    model,
    optimizer=optimizer,
    scheduler=scheduler,
    label_smooth=True,
    use_gpu=torch.cuda.is_available()
)

# Move model to GPU if available
if torch.cuda.is_available():
    model = model.cuda()

engine.run(
    save_dir='og/osnet',
    max_epoch=1,
    eval_freq=1,
    print_freq=2,
    test_only=False
)

=> Start training
epoch: [1/1][2/41]	time 0.319 (6.580)	data 0.001 (5.349)	eta 0:04:16	loss 2.8605 (2.8960)	acc 15.6250 (10.9375)	lr 0.000300
epoch: [1/1][4/41]	time 1.526 (5.821)	data 1.208 (5.032)	eta 0:03:35	loss 2.7176 (2.8327)	acc 18.7500 (14.8438)	lr 0.000300
epoch: [1/1][6/41]	time 1.953 (5.645)	data 1.638 (5.003)	eta 0:03:17	loss 2.5214 (2.7304)	acc 46.8750 (27.0833)	lr 0.000300
epoch: [1/1][8/41]	time 0.593 (5.544)	data 0.275 (4.974)	eta 0:03:02	loss 2.3996 (2.6673)	acc 40.6250 (29.6875)	lr 0.000300
epoch: [1/1][10/41]	time 0.836 (5.504)	data 0.518 (4.977)	eta 0:02:50	loss 2.2792 (2.5865)	acc 50.0000 (34.6875)	lr 0.000300
epoch: [1/1][12/41]	time 0.321 (5.562)	data 0.000 (5.063)	eta 0:02:41	loss 2.1032 (2.5063)	acc 56.2500 (38.5417)	lr 0.000300
epoch: [1/1][14/41]	time 0.320 (5.487)	data 0.000 (5.010)	eta 0:02:28	loss 2.0110 (2.4357)	acc 65.6250 (42.4107)	lr 0.000300
epoch: [1/1][16/41]	time 0.326 (5.429)	data 0.001 (4.968)	eta 0:02:15	loss 1.9423 (2.3822)	acc 68.7500 (45.1172

In [ ]:
!cp -r og /content/drive/MyDrive/stealth\ mode/
